# Steel Plates Fault Analysis

The goal of this document is to describe the approach used to analyse the dataset related to the Steel Plates Faults at the following [link](http://archive.ics.uci.edu/ml/datasets/steel+plates+faults).

The dataset includes 1941 observations, and 27 features. The data is already labeled, and there are 7 types of steel plate faults that are added to the dataset as 7 fields representing the one-hot-encoding of the label.

## Installations and Usage Giudelines

For this project I have been using the Anaconda distribution version 1.8.7 with python version 3.6.5. However, the normal installation of python should also work.

Here you may find the required modules that have been used in this project. The codes are written in Python 3.

In [ ]:
import numpy
import pandas
import sklearn
import scipy
import collections
import matplotlib
import seaborn
import sys

In [ ]:
modules = list(set(sys.modules) & set(globals()))
for module_name in modules:
    module = sys.modules[module_name]
    print(module_name, getattr(module, '__version__', 'unknown'))

For the complete list of requirements and their dependencies please take a look at the __requirements.txt__ file provided. To replicate my exact environment in anaconda distribution you may use the following command using the __machine-learning.yml__ file provided:

    conda env create -f machine-learnin.yml



## The Analysis

I am going to start by importing the relevant libraries and the dataset. Here Numpy and Pandas libraries are used for computations and handling the dataset. Scikit-Learn has been utilized for the machine learning methods that are used for analyzing the data, and Scipy has provided some more advanced statistical tools that were needed for this analysis.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
from scipy import stats
from scipy.stats import norm, skew

from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

Let's read the dataset and look inside:

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
data = pd.read_csv('../input/faulty-steel-plates/faults.csv')

From the data documentation we how that we have no missing values in our dataset. Therefore, there is no need to do the fixing for missing values.

Now that the dataset is created let's take a look at the first 5 rows to see if the data is ok:

In [ ]:
data.head()

At the moment the data is consist of 1941 observations and 32 columns from which 7 are the encoded fault types, we can also look at the descriptive statistics on the dataset:

In [ ]:
data.shape

In [ ]:
data.describe()

To be able to use the data, I will drop the one-hot encoding of the types and add a single __Target__ column with the fault type names:

In [ ]:
targets = data.iloc[:, 27:35]
data.drop(targets.columns, axis=1, inplace=True)
data['Target'] = targets.idxmax(1)
data.head()

Let's create a copy of the data that remains unchanged with the pre-procecssing:

In [ ]:
origina_data = data.copy()

Let's take a look at the classes of faults:

In [ ]:
target_counts= data['Target'].value_counts()

fig, ax = plt.subplots(1, 2, figsize=(15,7))
target_counts_barplot = sns.barplot(x = target_counts.index,y = target_counts.values, ax = ax[0])
target_counts_barplot.set_ylabel('Number of classes in the dataset')

colors = ['#8d99ae','#ffe066', '#f77f00','#348aa7','#bce784','#ffcc99',  '#f25f5c']
target_counts.plot.pie(autopct="%1.1f%%", ax=ax[1], colors=colors)


There are 7 fault classes as follows: Dirtiness, Stains, Pastry, Z_Scratch, K_Scratch, Bumps, and Other_Faults.

It can be seen that there are many observations (34.7%) that are labeled as __Other_Faults__. And besides __Bumps__, and __k_Scatch__ that cover considerable number of observations, other fault types do not have very high number of observations. This shows that in this set of observations the other four fault types do not accure often. However, this might cause an issue as classification methods might not perform well in recognizing the less populated faults. Therefore, a balanced sampling method might provide some improvements in the performance of the classification method. However, it should be studied and tested to give a final opinion.

In the following, I would like to look at the pairplot to take a look at the distribution of the features. The pairplot will be very crowded as there are many features to be examined. However, with zooming in it is possible to get some ideas:

In [ ]:
sns.pairplot(data, hue='Target')

From the pair plot I can see that the _TypeOfSteel_A300_, _TypeOfSteel_A400_ and _Outside_Global_Index_ are actually categorical featrues:

In [ ]:
data['TypeOfSteel_A300'] = data['TypeOfSteel_A300'].astype('category',copy=False)
data['TypeOfSteel_A400'] = data['TypeOfSteel_A400'].astype('category',copy=False)
data['Outside_Global_Index'] = data['Outside_Global_Index'].astype('category',copy=False)

Also, I could notice some correlations between some of the features. To investigate it more deeply I will use Peason's Correlation of the numerical features to see if the features are mutually independent. I use a heatmap to visualize the correlation:

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(data.corr(), cmap='seismic')

From the heatmap, it can be seen that there are some features that are highly correlated (shown in dark red for positive correlation and dark blue for negative correlation), with each other and cannot be considered as indipendent variables that we need for modeling. The features with white or very light colors however, can be condidered as indipendent. Let's examine some of the examples of correlated features below:

In [ ]:
sns.regplot(x='X_Minimum', y='X_Maximum', data = data, scatter = True)

In [ ]:
sns.regplot(x='Pixels_Areas', y='Sum_of_Luminosity', data = data, scatter = True)

In [ ]:
sns.regplot(x='Maximum_of_Luminosity', y='Luminosity_Index', data = data, scatter = True)

In [ ]:
sns.regplot(x='Edges_Y_Index', y='Log_X_Index', data = data, scatter = True)

The linear relationship between the two features shows their correlation. To resolve this issue the Principal Component Analysis (PCA) method might come handy to produce the indipendent components. However, the issue with this method is that the results of the classification based on PCA components is not easily interpretable based on the features.

## Feature Skewness

When we talk about normality what we mean is that the data should look like a normal distribution. This is important because several statistic tests rely on this (e.g. t-statistics).

If the dataset is skewed, then the Machine Learning model wouldn’t be able to do a good job on predictions. To resolve the issue of the skewed features we can apply the a log transform of the same data, or to use the Box-Cox Transformation. Let's see how skewed are our numerical features:

In [ ]:
numeric_features = data.dtypes[data.dtypes != "object"].index

skewed_features = data[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)

skewed_features_df = pd.DataFrame(skewed_features, columns={'Skew'})
skewed_features_df.head(10)

In [ ]:
skewed_features_df.tail(10)

I remove our categorical features from the list before applying the Box-Cox transformation:

In [ ]:
skewed_features_df.drop(['TypeOfSteel_A400','TypeOfSteel_A300', 'Outside_Global_Index'], inplace=True)

From what we can see, there are many features that are skewed, therefore, a method to resolve this issue might provide better performances in many of the models that would be applied on this dataset. Let's see one of the skewed features and compare the results after applying the Box-Cox Transformation:

In [ ]:
sns.distplot(data['Sum_of_Luminosity'])

Now I apply the Box-Cox transformation that is available in scipy special module:

In [ ]:
skewed_features_df = skewed_features_df[abs(skewed_features_df) > 0.75]

from scipy.special import boxcox1p
lam = 0.15
cols = skewed_features_df.index

for c in cols:
    data[c] = boxcox1p(data[c], lam)

In [ ]:
sns.distplot(data['Sum_of_Luminosity'])

It can be seen that the sample feature that we examined is now very close to a normal distribution.

## Scaling the Features

In many models the features need to be scaled for the model to perform well. Features with high values may effect the performance of many models. I will first separate the features and the target from our dataset and then apply the standard scaler on the features:

In [ ]:
features = data.drop('Target', axis=1)
target = data['Target']

scaler = StandardScaler()
features_scaled = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

## Outlier Analysis

From the pairplot that was examined previously, I also noticed many outliers. There are basically three types of outliers:
* __Point Outliers:__ observations anomalous with respect to the majority of observations in a feature (aka univariate outlier).
* __Contextual Outliers:__ observations considered anomalous given a specific context.
* __Collective Outliers:__ a collection of observations anomalous but appear close to one another because they all have a similar anomalous value.

In what follows I try to analyse the outliers and remove them if possible. 

In a more thorough analysis it would be useful to utilize the expert's knowledge to understand if the found outliers should be actually removed or they are revealing useful information.

### Univariate Method:

Tukey Boxplot is used to detect unusually high or low data points, which can be considered as potential outliers to be investigated further. Even if it would be very crowded, I would like to take a look of the boxplot of the all features to get an idea about how many outliers we are talking about:

In [ ]:
data_boxplot = features_scaled.boxplot(return_type='dict', vert=False, figsize=(20,20))

It seems that there are many outliers in different features that is expected as we are looking at the fault data. However, the outliers should be studied, and if they are real outliers they should be removed. Let's first look at the extreme outliers:

In [ ]:
features_scaled[features_scaled['Pixels_Areas']>4]

In [ ]:
features_scaled[features_scaled['Sum_of_Luminosity']>4]

In [ ]:
features_scaled[features_scaled['X_Perimeter']>4]

In [ ]:
features_scaled[features_scaled['Y_Perimeter']>4]

It seems that these outliers are related to the same observations. This is not very suprising as we have seen previously that these 4 features are highly correlated. 

The univariate study shows that there are many outliers, however, more study is required to decide how to manage them. Therefore, it should be taken into consideration in a more thorough analysis.

### Multivariate Method

I would also like to do a multivariate analysis to reveal the outliers in the features with regards to the target classes. 

To go forward with the analysis, I create the scaled dataset from the scaled features and I add the target, and also a numeric code to represent the target class:

In [ ]:
data_scaled = features_scaled.copy()
data_scaled['Target'] = target

data_scaled['Target'] = pd.Categorical(data_scaled['Target'])
data_scaled['Target_Code'] = data_scaled.Target.cat.codes

After some try and error I selected some boxplots that will give some information about the amount of outliers that we are facing:

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Target', y='X_Maximum', data=data_scaled)

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Target', y='Steel_Plate_Thickness', data=data_scaled)

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Target', y='Luminosity_Index', data=data_scaled)

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Target', y='Square_Index', data=data_scaled)

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Target', y='Edges_Index', data=data_scaled)

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Target', y='LogOfAreas', data=data_scaled)

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Target', y='Y_Maximum', data=data_scaled)

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Target', y='Orientation_Index', data=data_scaled)

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Target', y='Minimum_of_Luminosity', data=data_scaled)

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Target', y='Length_of_Conveyer', data=data_scaled)

As it can be seen in the plots, there are many outliers, including some very extreme ones that would reduce the performance of the models. As in the data documentations they aren't enough information on the features and expert knowledge cannot not be accessed at the current time, it is not possible to examine the outliers to decide which ones can be removed. In the following, I will use DBSCAN as a method to remove the outliers in a more automatic way:

### DBSCAN for Outlier Detection

DBSCAN is a unsupervised method that clusters core samples and denotes non-core samples. It can be used to identify collective outliers. I start from $eps =0.1$ and increase it untill I arrive to maximum 5% of the dataset to be considered as outliers. _eps_ is the maximum distance between two samples for them to be considered as in the same neighborhood. I also play with the _min_samples_ parameter which defiens the minimum number of the samples in a cluster.

In [ ]:
dbscan_model = DBSCAN(eps=3.3, min_samples=7).fit(features_scaled)

In [ ]:
print(Counter(dbscan_model.labels_))

Here the DBSCAN has found 95 outliers which is about 4.89% of our dataset which is acceptable as we should remain under 5% when detecting the outliers. This is not the optimal output that was expected, as it wasn't possible to detect more clusters and there was always too many observations considered as outliers.

As there aren't any other information to decide about the outliers I will simply drop the outliers from the dataset:

In [ ]:
outliers = features_scaled[dbscan_model.labels_ == -1]
outliers.shape

In [ ]:
features_scaled.drop(outliers.index, axis=0, inplace=True)
target.drop(outliers.index, axis=0, inplace=True)
data_scaled.drop(outliers.index, axis=0, inplace=True)
features_scaled.shape

## Principal Component Analysis

As it has been discusssed before, there are many correlated variables in the dataset. Therefore, PCA might help in creating independent components that can be used in the final models.

Here I try to run the PCA to create components representing all the features. I will use this to select the right number of components that better represent the variation in the dataset:

In [ ]:
pca = PCA(random_state=101)
features_pca = pca.fit_transform(features_scaled.values)
pd.DataFrame(pca.explained_variance_ratio_, columns=['Explained Variance Ratio'])

In [ ]:
pca.explained_variance_ratio_[0:15].sum()

It seems that we can explain the 99% of the variability in the data set using the first 15 components from the PCA.

Let's see how the PCA components correlate with our featuers:

In [ ]:
pca_components = pd.DataFrame(pca.components_, columns= features.columns)
plt.figure(figsize=(20,20))
sns.heatmap(pca_components, cmap='seismic')

Now I choose the number of components to 15, and run the PCA again to create the PCA components:

In [ ]:
def pca_dataset(features, n_components):
    
    pca_n = PCA(n_components=n_components, random_state=101)
    features_pca_n = pca_n.fit_transform(features)
    
    column_pca = []
    for i in range(0,n_components):
        column_pca.append('Component'+np.str(i))
    return pd.DataFrame(features_pca_n, columns=column_pca)

I create the dataset based on the results, I simply call the columns 'Component0'...'Component14':

In [ ]:
data_pca15 = pca_dataset(features_scaled, n_components=15)
data_pca15['Target'] = target

Let's run the paiplot one more time to see how the components are distributed:

In [ ]:
sns.pairplot(data_pca15, hue='Target')

Here we can see the components are mutually independent from one another. However, still it seems like the last components do not play an important role in separating the classes. I will also create another dataset using only 5 components that make up to the 76% variability as can be seen below:

In [ ]:
pca.explained_variance_ratio_[0:5].sum()

In [ ]:
data_pca5 = pca_dataset(features_scaled, n_components=5)
data_pca5['Target'] = target

I will also add the target codes to the created dataset which will be useful later on:

In [ ]:
data_pca15['Target'] = pd.Categorical(data_pca15['Target'])
data_pca15['Target_Code'] = data_pca15.Target.cat.codes

data_pca5['Target'] = pd.Categorical(data_pca5['Target'])
data_pca5['Target_Code'] = data_pca5.Target.cat.codes

## Cluster Analysis

### Grouping with K-Means Clustering

The K-Means clustering algorithm is a simple unsupervised algorithm that's used for quickly predicting groupings.

Now let's take a look at the groups that we can identify by K-Means. We already know that we have 7 types of fault. Therefore, we can consider the number of clusters to be 7.

In [ ]:
kmeans_model = KMeans(n_clusters=7, random_state=54)
kmeans_model.fit(features_scaled)

To see the results, I will choose one example features and we can compare how the ground truth classification (with the labels) will compare to the clusters that are recognized with the K-Means:

In [ ]:
kmeans_labels = np.choose(kmeans_model.labels_, [0,1,2,3,4,5,6]).astype(np.int64)
data_scaled['kmeans_labels'] = kmeans_labels

In [ ]:
color_themes = {0:'#8d99ae',1:'#ffe066', 2:'#f77f00',3:'#348aa7',4:'#bce784',5:'#ffcc99',  6:'#f25f5c'}


sns.lmplot(x='Orientation_Index', y='Log_X_Index', data=data_scaled, fit_reg=False, hue='Target', col='Target', size=8)
plt.title("Ground Truth Classification")

sns.lmplot(x='Orientation_Index', y='Log_X_Index', data=data_scaled,  fit_reg=False, hue='kmeans_labels', col='kmeans_labels',size=8)
plt.title("KMean Clustering")

From this example we can see that K-Means could some how find the right clusters however it doesn't seem close enough. Let's look at the classification metrics to get a more detailed evaluation:

In [ ]:
print(classification_report(data_scaled['Target_Code'], kmeans_labels))

As the results show the precision and recall suffer, and K-Means is not able to cluster the data accurately. Let's try the same approach using the 15 PCA components that we selected:

In [ ]:
kmeans_model_pca15 = KMeans(n_clusters=7, random_state=54)
kmeans_model_pca15.fit(data_pca15.drop(['Target','Target_Code'], axis=1))

In [ ]:
kmeans_labels_pca15 = np.choose(kmeans_model.labels_, [0,1,2,3,4,5,6]).astype(np.int64)
data_pca15['kmeans_labels'] = kmeans_labels_pca15

In [ ]:
sns.lmplot(x='Component0', y='Component1', data=data_pca15, fit_reg=False, hue='Target', col='Target', size=8)
plt.title("Ground Truth Classification")

sns.lmplot(x='Component0', y='Component1', data=data_pca15,  fit_reg=False, hue='kmeans_labels', col='kmeans_labels',size=8)
plt.title("KMean Clustering")

In [ ]:
print(classification_report(data_pca15['Target_Code'], kmeans_model_pca15.labels_))

It can be seen from the results that the 15 PCA components have led to much better precision and recall in K-Means. Let's also try with the 5 PCA components:

In [ ]:
kmeans_model_pca5 = KMeans(n_clusters=7, random_state=54)
kmeans_model_pca5.fit(data_pca5.drop(['Target','Target_Code'], axis=1))

In [ ]:
print(classification_report(data_pca5['Target_Code'], kmeans_model_pca15.labels_))

We can see the using the 5 PCA components we arrive at the same results. Therefore, it can be considered to use less components would not impact the performance of the model negatively. Let's now try another clustering method that might lead to better results compared to K-Means:

### Hierarchical Clustering Method

Hierarchical clustering methods predict subgroups within data by finding the distance between each data point and its nearest neighbors. As we have irregular classes that are not normally distributed, this method might have an advantage to K-Means in producing more accurate clusters.

However, as the original shape of the distribution is important here and this model doesn't perform well on normal distributions, I will use the original dataset without the transformations and scaling for this model.

First, I use scipy yo generate dendograms:

In [ ]:
original_features = origina_data.drop(['Target'], axis=1).copy()
origina_data['Target'] = pd.Categorical(origina_data['Target'])
origina_data['Target_Code'] = origina_data.Target.cat.codes

In [ ]:
linkage_model = linkage(original_features, method='ward')
dendrogram(linkage_model, truncate_mode='lastp', p=12, leaf_rotation=45, leaf_font_size=12, show_contracted=True)
plt.title('Truncated Hierarchical Clustering Dendrogram')
plt.xlabel('Cluster Size')
plt.ylabel('Distance')

plt.axhline(y=0.4*10**(8))
plt.axhline(y=0.2*10**(8))

As it can be seen in the dendrogram, the closest that we can arrive to the desired 7 clusters is if we deside to have the distance around 0.2le8 which is quite strange. Let's see how the hierarchical clustering will perform here:

### Generating hierarchical clusters

In [ ]:
k = 7
h_clustering = AgglomerativeClustering(n_clusters=k, affinity='euclidean', linkage='ward')
h_clustering.fit(original_features)

accuracy_score(origina_data['Target_Code'], h_clustering.labels_)

In [ ]:
h_clustering = AgglomerativeClustering(n_clusters=k, affinity='manhattan', linkage='complete' )
h_clustering.fit(original_features)

accuracy_score(origina_data['Target_Code'], h_clustering.labels_)

In [ ]:
h_clustering = AgglomerativeClustering(n_clusters=k, affinity='manhattan', linkage='average')
h_clustering.fit(original_features)

accuracy_score(origina_data['Target_Code'], h_clustering.labels_)

After some try and error, it seems like the distance metric Euclidean and the linkage "Ward" work the best on our data, however, the results are still far from desired. Let's also use the PCA components and see if we can get better results:

In [ ]:
k = 7
h_clustering_pca5 = AgglomerativeClustering(n_clusters=k, affinity='euclidean', linkage='ward' )
h_clustering_pca5.fit(data_pca5.drop(['Target','Target_Code'], axis=1))

accuracy_score(data_pca5['Target_Code'], h_clustering_pca5.labels_)

Using the 5 selected components of the PCA method, it can be seen the results are far from satisfying.